In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import xarray as xr

import esmtools as et
from esmtools import stats

import shapefile
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geopandas as gp
import regionmask
import matplotlib.patches as mpatches

In [2]:
# Open dataset 
dir = '/chinook/dallmann/SOARS/'
fn = 'tos_Omon_MRI-ESM2-0_omip2_r1i1p1f1_gr_165301-201812.nc'

data_tos = xr.open_dataset(dir+fn)
data_tos

/opt/miniconda3/envs/prod/lib/python3.8/site-packages/xarray/coding/times.py:527: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/opt/miniconda3/envs/prod/lib/python3.8/site-packages/numpy/core/_asarray.py:83: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)


<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 180, lon: 360, time: 4392)
Coordinates:
  * time       (time) object 1653-01-16 12:00:00 ... 2018-12-16 12:00:00
  * lat        (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * lon        (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object ...
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    tos        (time, lat, lon) float32 ...
Attributes: (12/36)
    Conventions:           CF-1.7 CMIP-6.2
    activity_id:           OMIP
    branch_method:         no parent
    creation_date:         2020-03-26T15:26:15Z
    data_specs_version:    01.00.31
    experiment:            OMIP experiment forced by JRA55-do atmospheric dat...
    ...                    ...
    title:                 MRI-ESM2-0 output prepared for CMIP6
    variable_id:           tos
    variant_label:         r1i1p1f1
    license:               CMIP6 model data produced by MRI is licensed under...
    cmor_version:          3.5.0
    tracking_id:           hdl:21.14100/8fdc222c-7031-46f7-8434-2bd763e70019

In [6]:
# Change the time to be from DatetimeNoLeap to a Datetime64
data_tos['time'] = np.arange('1958-01-01','2019-01-01',dtype = 'datetime64[M]')
data_tos

In [7]:
# select dimensions!
data_tos['time'] = pd.date_range("1958-01", "2018-12", freq="MS")
tos = data_tos['tos']
tos = tos.sel(lon=slice(190,270),lat=slice(-10,70))

ValueError: conflicting sizes for dimension 'time': length 4392 on 'time_bnds' and length 732 on {'time': 'time'}

In [ ]:
# remove long term trend for each grid cell
var = stats.rm_poly(tos,order=2,dim='time',nan_policy='drop')
var

In [ ]:
# 1982-2010 period
base = var.sel(time=slice('1982-01','2010-12'))
tos_base = base
tos_base

In [ ]:
# Blob period
blob = var.sel(time=slice('2013-07','2016-06'))
tos_blob = blob
tos_blob

In [ ]:
# Difference between the two periods
blob_anom = blob.groupby('time.month') - base.groupby('time.month').mean()
blob_anom

In [ ]:
# Resample to two month resolution
fosi = blob_anom.resample(time='2M').mean()
fosi

In [ ]:
def panels(data,description,variable,depth,lower_bound,upper_bound, step,cmap):
    f, axs = plt.subplots(nrows=3,ncols=6,figsize=(20,6),
                          subplot_kw=dict(projection=ccrs.PlateCarree(central_longitude=180))) 
    axs = axs.flatten()
    for i in range(0,18):
        im = axs[i].contourf(fosi.lon, fosi.lat, fosi.isel(time=i),
                         transform=ccrs.PlateCarree(), 
                         cmap=cmap,
                         levels=np.arange(lower_bound, upper_bound, step),
                         extend='both')
        axs[i].add_feature(cfeature.LAND, color='k')
        axs[i].add_patch(mpatches.Rectangle(xy=[210, 40], width=15, height=10,
                                    edgecolor='k',
                                    facecolor='none',
                                    transform=ccrs.PlateCarree())
                 )
    plt.suptitle("FOSI: " + description + " " + variable + " " + depth + " (de-trended)")
    axs[0].set_title("Jul-Aug"); axs[1].set_title("Sept-Oct")
    axs[2].set_title("Nov-Dec"); axs[3].set_title("Jan-Feb")
    axs[4].set_title("Mar-Apr"); axs[5].set_title("May-Jun")
    plt.text(-790,215,'2013-14')
    plt.text(-790,120,'2014-15')
    plt.text(-790,25,'2015-16')
    f.subplots_adjust(right=0.8)
    cbar_ax = f.add_axes([0.82, 0.15, 0.05, 0.7])
    f.colorbar(im, cax=cbar_ax,fraction=0.046,pad=0.04)
    
panels(tos,'SSTA','NE','Pacific',-4.0,4.0, 1.0,'RdBu_r')